In [4]:
import requests
import pandas as pd
import os
import json
import time
from requests.exceptions import SSLError, RequestException  # RequestException 是所有请求异常的基类

In [5]:
http_url = "https://api-cn.faceplusplus.com/facepp/v3/detect" 
key ="apD0OPgriGmRItHsYwbLIT1GFmtHV0zU"
secret ="oMW9bU7R-HHw4665UkmCKHCwwxxlhu_1"
data = {"api_key": key, "api_secret": secret, "return_landmark": "1","return_attributes":"beauty"}

In [6]:
manager_scores = []
error_list = []
for root, dirs, files in os.walk("/Users/lzc19950714/PycharmProjects/pythonProject/ADA/AMAC_photos"):
    print(f"共包括{len(files)}名基金经理照片")
    for pic_name in files:
        company_name = pic_name[:-4].split('_')[0]
        manager_name = pic_name[:-4].split('_')[1]
        manager_score = {'company_name': company_name, 'manager_name': manager_name}
        file_path = os.path.join(root, pic_name)
        pics = {"image_file": open(file_path, "rb")}
        time.sleep(0.05)
        try:
            response = requests.post(http_url, data=data, files=pics)
            if response.status_code == 200:
                content = response.content.decode('utf-8')
                res_dict = json.JSONDecoder().decode(content)
                score = res_dict['faces'][0]['attributes']['beauty']
            else:
                score = {'male_score': None, 'female_score': None}
            manager_score.update(score)
            manager_scores.append(manager_score)
        except SSLError as ssl_err:
            print(f"SSL 错误: {ssl_err}\n")
            print(pic_name)
            error_list.append(manager_name)
        except RequestException as req_err:
            print(f"请求错误: {req_err}\n")
            print(pic_name)
            error_list.append(manager_name)
        except Exception as e:
            print(f"发生了一个错误: {e}\n")
            print(pic_name)
            error_list.append(manager_name)

共包括3485名基金经理照片
发生了一个错误: list index out of range

华夏基金管理有限公司_何家琪.jpg
发生了一个错误: list index out of range

招商基金管理有限公司_许荣漫.jpg
发生了一个错误: list index out of range

南方基金管理股份有限公司_刘益成.jpg
发生了一个错误: list index out of range

施罗德基金管理（中国）有限公司_单坤.jpg
发生了一个错误: list index out of range

上银基金管理有限公司_鉏国彬.jpg
发生了一个错误: list index out of range

嘉实基金管理有限公司_张宇驰.jpg
发生了一个错误: list index out of range

中邮创业基金管理股份有限公司_王高.jpg
发生了一个错误: list index out of range

农银汇理基金管理有限公司_张燕.jpg
发生了一个错误: list index out of range

泓德基金管理有限公司_胡云.jpg
发生了一个错误: list index out of range

国海富兰克林基金管理有限公司_沈竹熙.jpg


In [7]:
manager_scores = pd.DataFrame(manager_scores)

In [17]:
managers_photo_list = pd.read_excel("/Users/lzc19950714/PycharmProjects/pythonProject/ADA/Manager_photo.xlsx", index_col=0)
managers_photo_list.reset_index(inplace=True, drop=True)

In [ ]:
managers_photo_list.columns = ["Manager_Id",	"manager_name", "company_name", "photo_urls"]
manager_scores = pd.merge(manager_scores, managers_photo_list, on=['manager_name', "company_name"], how='inner')

In [23]:
manager_scores_output = manager_scores[['manager_name', 'company_name', 'male_score', 'female_score', 'photo_urls']]

In [26]:
manager_scores_output.to_csv('/Users/lzc19950714/PycharmProjects/pythonProject/ADA/Manager_Scores.csv', encoding='utf-8')